# Gauss Elimination
Linear algebraic equations are represented as:
$$
\begin{bmatrix} A \end{bmatrix}
\begin{Bmatrix} x \end{Bmatrix} = \begin{Bmatrix} B \end{Bmatrix}
$$
Given $[A]$ and $\{ B \}$, we must find $\{ x \}$. Methods of solving linear algebraic equations can be categorized into:
1. **Direct Methods:** which are non-iterative and involve a fixed number of steps
2. **Iterative Methods:** which are iterative and repeat a fixed sequence of steps agai and again until the results reach the desired accuracy.

Gauss elimination is a direct method and serves as the starting point for many other direct methods.

Gauss elimination method involves a series of row operations carried out on the coefficient matrix and the right hand side vector such that the coefficient matrix is converted into an upper triangular matrix. Solving linear algebraic equations when coefficient matrix is an upper triangular matrix is trivial, as the last row contains only one unknown and can be easily solved. Subsequently, working backwards one row at a time, we can solve for the remaining unknowns as each row now contains only one unknown while the others are determined from the rows below it.

Gauss elimination consists of two steps:
1. Forward elimination
2. Back substitution

## Naive Gauss Elimination
Pivoting is the process of interchanging the rows to bring the row with largest absolute value in the pivotal column to the pivotal position. Pivoting increases numerical accuracy of the solution as well as make it possible to solve some problems which cannot be solved without pivoting. At this point of time, we will not consider pivoting and naively carry out Gauss elimination.

We will first input the given data and solve the equations using the built-in function **``numpy.linalg.solve()``**

In [1]:
from __future__ import division
import numpy as np

# Input coefficient matrix [A] and right hand side vector {b}
a = np.array([ [5, 4, 1], [10, 9, 4], [10, 13, 15]], dtype=float)
print 'Before Forward Elimination'
print a
b = np.array([6.8, 17.6, 38.4])
print b

# Solve the equations using built-in function numpy.linalg.solve()
x = np.linalg.solve(a, b)
print
print 'Solution'
print x
print np.dot(a, x) - b

Before Forward Elimination
[[  5.   4.   1.]
 [ 10.   9.   4.]
 [ 10.  13.  15.]]
[  6.8  17.6  38.4]

Solution
[ 0.4  0.8  1.6]
[  0.00000000e+00   0.00000000e+00  -7.10542736e-15]


## Count Rows and Columns and Loop through Rows 
We will first learn how to count the number of rows and columns and then loop through the rows, select the pivotal row and determne the rows that will be modified for a given pivotal row.

In [2]:
m, n = a.shape
print 'Rows =', m, 'Columns =', n
for i in range(m):
    print 'Row', i, ':', a[i, :], b[i]

for p in range(m-1):
    print 'Pivotal row', p
    for i in range(p+1, m):
        f = a[i,p] / a[p, p]
        print '\t', 'Row:', i, 'Factor', f, a[i,:], b[i]

Rows = 3 Columns = 3
Row 0 : [ 5.  4.  1.] 6.8
Row 1 : [ 10.   9.   4.] 17.6
Row 2 : [ 10.  13.  15.] 38.4
Pivotal row 0
	Row: 1 Factor 2.0 [ 10.   9.   4.] 17.6
	Row: 2 Factor 2.0 [ 10.  13.  15.] 38.4
Pivotal row 1
	Row: 2 Factor 1.44444444444 [ 10.  13.  15.] 38.4


### Function for Forward Elimination
Having understood how to count rows and columns and selecting pivotal row and rows to be modified for each pivotal row, we can now proceed to carry out naive forward elimination. Forward elimination is a series of row operations, mainly involving subtraction of scaled form of the pivotal row from the row being modified such that the element in the pivotal column of the row being modified must become zero. At the end of forward elimination, coefficient matrix $[A]$ must be converted into an upper triangular matrix and right hand side vector $\{ b \}$ must be modified compared to the original.

In [3]:
def forward_elim(a, b):
    m, n = a.shape
    if m != n:
        print 'Coefficient matrix [A] must be square'
        return
    if m != len(b):
        print 'Number of rows in [A] must equal number of elements in {b}'
        return
    for p in range(m-1):
        for i in range(p+1, m):
            f = a[i, p] / a[p, p]
            a[i, p:] = a[i, p:] - f * a[p, p:]
            b[i] = b[i] - f * b[p]
    return a, b

aa, bb = forward_elim(a, b)
print 'After Forward Elimination'
print aa
print bb

After Forward Elimination
[[ 5.  4.  1.]
 [ 0.  1.  2.]
 [ 0.  0.  3.]]
[ 6.8  4.   4.8]


## Back Substitution
When coeficient matrix $[A]$ is an upper triangular matrix, the last unknown $x_n$ can be obtained easily because all elements in the last row of $[A]$ except $a_{nn}$ are zero. Thus
$$x_n = \frac{b_n}{a_{nn}}$$
With $x_n$ known, we can gradually work backwards and find the remaining unknowns in the reverse sequence $x_{n-1}, x_{n-1}, \ldots, x_n$.

To find the unknown $x_i$ corresponding to row $i$, we must multiply columns of that row with the corresponding unknowns that have been previously determined and subtract it from the value on the right hand side. Then we must divide that difference with the diagonal element of that row.
$$x_i = \frac{b_i - \sum_{j=i+1}^{n}{a_{ij} x_j}}{a_{ii}}$$
### Function for Back Substitution

In [4]:
x = np.zeros((m,), dtype=float)
print x

x[-1] = bb[-1] / aa[-1, -1]
print x

for i in range(m-2, -1, -1):
    s = np.dot(aa[i, i+1:], x[i+1:])
    print 'Row', i, aa[i, i+1:], x[i+1:], bb[i], 'Sum =', s
    x[i] = (bb[i] - s) / aa[i, i]
    print x[i]
print
print 'Solution'
print x

[ 0.  0.  0.]
[ 0.   0.   1.6]
Row 1 [ 2.] [ 1.6] 4.0 Sum = 3.2
0.8
Row 0 [ 4.  1.] [ 0.8  1.6] 6.8 Sum = 4.8
0.4

Solution
[ 0.4  0.8  1.6]


We can now convert this procedure into a function that takes in the modified coefficient matrix $[A]$ and right hand side vector $\{ b \}$ obtained after forward elimination and calculates all the unknowns $\{ x \}$.

In [5]:
def back_sub(a, b):
    m, n = a.shape
    x = np.zeros_like(b) #(m, dtype=float)
    x[-1] = b[-1] / a[-1, -1]
    for i in range(m-1, -1, -1):
        s = np.dot(a[i,i+1:], x[i+1:])
        x[i] = (b[i] - s) / a[i, i]
    return x

x = back_sub(aa, bb)
print 'Solution'
print x
print np.dot(a, x) - b

Solution
[ 0.4  0.8  1.6]
[ 0.  0.  0.]


## Partial Pivoting
Before commencing with modifying the rows below the pivotal row, we must examine which of the rows is best suited to be the pivotal row. The row with the largest absolute value in the pivotal column must be exchanged with the pivotal row. We must therefore be able to identify row number of the row with the largest absolute value in the pivotal column, starting from the pivotal row.
$$max(a_{ip}), \qquad i=p, p+1, \ldots, n$$

In [6]:
def partial_pivoting(a, b, p):
    m, n = a.shape
    amax = np.abs(a[p, p])
    imax = p
    for i in range(p+1, n):
        if np.abs(a[i, p] > amax):
            imax = i
            amax = np.abs(a[i, p])
    if imax > p:
        tmp = a[p, p:].copy()
        a[p, p:] = a[imax, p:].copy()
        a[imax, p:] = tmp.copy()
        
        tmp = b[p]
        b[p] = b[imax]
        b[imax] = tmp
    return a, b

a = np.array([ [0, 2, 3, 9], [4, 2, 4, 0], [2, 2, 3, 2], [4, 3, 6, 3] ], dtype=float)
b = np.array([122, 20, 36, 60], dtype=float)
print 'Before partial pivoting'
print a
print b

Before partial pivoting
[[ 0.  2.  3.  9.]
 [ 4.  2.  4.  0.]
 [ 2.  2.  3.  2.]
 [ 4.  3.  6.  3.]]
[ 122.   20.   36.   60.]


In [7]:
aa, bb = partial_pivoting(a, b, 0)
print 'After partial pivoting'
print aa
print bb

After partial pivoting
[[ 4.  2.  4.  0.]
 [ 0.  2.  3.  9.]
 [ 2.  2.  3.  2.]
 [ 4.  3.  6.  3.]]
[  20.  122.   36.   60.]


## Forward Elimination with Partial Pivoting
We will now modify forward elimination in such a way that we can choose to use partial pivoting only when we want.

In [8]:
def forward_elim(a, b, pivoting=True):
    m, n = a.shape
    if m != n:
        print 'Coefficient matrix [A] must be square'
        return
    if m != len(b):
        print 'Number of rows in [A] must equal number of elements in {b}'
        return
    for p in range(m-1):
        if pivoting:
            a, b = partial_pivoting(a, b, p)
        for i in range(p+1, m):
            f = a[i, p] / a[p, p]
            a[i, p:] = a[i, p:] - f * a[p, p:]
            b[i] = b[i] - f * b[p]
    return a, b

a = np.array([ [0, 2, 3, 9], [4, 2, 4, 0], [2, 2, 3, 2], [4, 3, 6, 3] ], dtype=float)
b = np.array([122, 20, 36, 60], dtype=float)
print a
print b
print 'Without partial pivoting'
aa, bb = forward_elim(a, b, False)
print aa, bb

[[ 0.  2.  3.  9.]
 [ 4.  2.  4.  0.]
 [ 2.  2.  3.  2.]
 [ 4.  3.  6.  3.]]
[ 122.   20.   36.   60.]
Without partial pivoting
[[  0.   2.   3.   9.]
 [ nan -inf -inf -inf]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]] [ 122.  -inf   nan   nan]


C:\Anaconda\lib\site-packages\ipykernel\__main__.py:13: RuntimeWarning: divide by zero encountered in double_scalars
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: invalid value encountered in multiply
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:13: RuntimeWarning: invalid value encountered in double_scalars


If the diagonal element is zero, Gauss elimination fails. A simple interchange of rows can overcome this problem. If after row interchange, the diagonal element is still zero, it means that the coefficient matrix is singular and cannot be solved.

Here is the solution if partial pivoting is carried out.

In [9]:
a = np.array([ [0, 2, 3, 9], [4, 2, 4, 0], [2, 2, 3, 2], [4, 3, 6, 3] ], dtype=float)
b = np.array([122, 20, 36, 60], dtype=float)
print a
print b
print 'With partial pivoting'
aa, bb = forward_elim(a, b)
print aa, bb
x = back_sub(aa, bb)
print 'Solution'
print x
print 'Verification'
print np.dot(a, x) - b
print
print 'Solution using numpy.linalg.solve()'
print np.linalg.solve(a, b)

[[ 0.  2.  3.  9.]
 [ 4.  2.  4.  0.]
 [ 2.  2.  3.  2.]
 [ 4.  3.  6.  3.]]
[ 122.   20.   36.   60.]
With partial pivoting
[[ 4.   2.   4.   0. ]
 [ 0.   2.   3.   9. ]
 [ 0.   0.  -0.5 -2.5]
 [ 0.   0.   0.  -4. ]] [  20.  122.  -35.  -56.]
Solution
[  6.  -2.  -0.  14.]
Verification
[ 0.  0.  0.  0.]

Solution using numpy.linalg.solve()
[  6.  -2.  -0.  14.]
